In [1]:
import numpy as np

## Derivative

```python
from utilities import differentiate
```

In [ ]:
class DerivativeError(Exception):
  """
  Raised when there is an error in calculation of derivative
  """

  def __init__(self, message):
    super().__init__(message)

def calculate_difference(points):
  """
  Returns a list of difference between two consecutive elements of the
  `points`.

  :param points: list of elements whose differences need to be returned
  :return: list of consectutive differences of elements in `points`

  Example:
  >>> points = [1,2,3,4]
  >>> calculate_difference(points)
  [1,1,1]

  # The array is computed as - [2-1, 3-2, 4-3]
  """

  d_points = []
  for i in range(0,len(points)-1):
    p1 = points[i]
    p2 = points[i+1]
    d_points.append(p2-p1)
  return d_points

def differentiate(x, y):
  """
  Differentiates y with respect to x.

  :param x: list of absicssae
  :param y: list of ordinates
  :return: derivative of y with respect to x

  .. math::
      \frac{dx}{dy} = lim_{\Delta x \to 0} \frac{\Delta y}{\Delta x}

  This function is effective when :math:`\Delta x \approx 0 `.

  Example:
  >>> x = [1,2,3,4,5]
  >>> y = [4,5,6,7,9]
  >>> differentiate(x, y)
  [1,1,1,2]
  """

  if len(x) != len(y):
    raise DerivativeError('Unequal datapoints in the dataset')
  x = calculate_difference(x)
  y = calculate_difference(y)
  i = 0
  consider = []
  while (i<len(x)):
    # neglect values where delta(x) = 0
    if x[i] == 0:
      x = x[:i] + x[i+1:]
      y = y[:i] + y[i+1:]
      continue
    consider.append(i)
    i+=1
  return x,  y, [y[i]/x[i] for i in range(len(x))], consider

## Group Duplicates

```python
from utilities import group_by
```

In [ ]:
def group_by(x, y):
  """
  Attemts to convert a relation from `x` to `y` to a function from `x` to `y`
  by grouping values of `y` as mean corresponding to same `x`.

  :param x: list of absicssae
  :param y: list of ordinates
  :return: a function from `x` to `y`

  Example:
  >>> x = [1,1,1,2,3]
  >>> y = [2,3,4,4,5]
  >>> group_by(x, y)
  ([1, 2, 3], [3, 4, 5])
  """

  new_x = []
  new_y = []
  datapoints = len(x)
  count = 1
  sum = 0
  for i in range(datapoints-1):
    if x[i] == x[i+1]:
      count += 1
      sum += y[i]
    elif x[i] != x[i+1]:
      new_x.append(x[i])
      if count == 1:
        new_y.append(y[i])
      else:
        new_y.append((sum+y[i])/count)
      count = 1
      sum = 0
  if x[datapoints - 1] == x[datapoints - 2]:
    new_x.append(x[datapoints - 2])
    new_y.append((sum+y[datapoints-1])/count)
  else:
    new_x.append(x[datapoints - 1])
    new_y.append(y[datapoints - 1]/count)
  return new_x, new_y

## Find nearest element

```python
from utilities import find_nearest
```

In [6]:
def find_nearest(array, value):
    """
    Returns the index in the array whose value is nearest to `value`
    :param array: list of values
    :param value: value to be found
    :return: the index in the array whose value is nearest to `value`
    """

    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx